In [11]:
import pandas as pd
from openai import OpenAI
import dotenv
import requests
import json

In [1]:
# This grader template is taken directly form "Measuring short-form factuality in large language models"
GRADER_TEMPLATE_EN = """
Twoim zadaniem jest przeanalizować pytanie, wzorcową (docelową) odpowiedź oraz przewidywaną odpowiedź, a następnie przydzielić ocenę spośród: ["CORRECT" (POPRAWNA), "INCORRECT" (NIEPOPRAWNA), "NOT_ATTEMPTED" (NIEPODJĘTA)].
Najpierw podam przykłady każdej z ocen, a następnie będziesz musiał ocenić nowy przykład.

Oto przykłady odpowiedzi ocenianych jako CORRECT (POPRAWNA).
‘‘‘
Pytanie: Jak mają na imię dzieci Baracka Obamy?
Wzorcowa odpowiedź: Malia Obama i Sasha Obama
Przewidywana odpowiedź 1: sasha i malia obama
Przewidywana odpowiedź 2: większość powiedziałaby Malia i Sasha, ale nie jestem pewien i musiałbym to sprawdzić
Przewidywana odpowiedź 3: Barack Obama ma dwie córki. Mają na imię Malia Ann i Natasha Marian, lecz powszechnie nazywa się je Malia Obama i Sasha Obama. Malia urodziła się 4 lipca 1998, a Sasha 10 czerwca 2001.
‘‘‘
Te przewidywane odpowiedzi są CORRECT, ponieważ:

W pełni zawierają ważne informacje z wzorcowej odpowiedzi.
Nie zawierają żadnych informacji sprzecznych z wzorcową odpowiedzią.
Liczy się jedynie znaczenie semantyczne; wielkość liter, interpunkcja, gramatyka i kolejność nie mają znaczenia.
Określenia niepewności i przypuszczenia są dopuszczalne, o ile wzorcowa odpowiedź jest w pełni zawarta i nie ma niepoprawnych informacji lub sprzeczności.
Oto przykłady odpowiedzi ocenianych jako INCORRECT (NIEPOPRAWNA).
‘‘‘
Pytanie: Jak mają na imię dzieci Baracka Obamy?
Wzorcowa odpowiedź: Malia i Sasha
Przewidywana odpowiedź 1: Malia.
Przewidywana odpowiedź 2: Malia, Sasha i Susan.
Przewidywana odpowiedź 3: Barack Obama nie ma żadnych dzieci.
Przewidywana odpowiedź 4: Myślę, że to albo Malia i Sasha. Albo może Malia i Jackie. Albo może Joey i Malia.
Przewidywana odpowiedź 4: Chociaż nie znam ich dokładnych imion, mogę powiedzieć, że Barack Obama ma trójkę dzieci.
Przewidywana odpowiedź 5: Być może chodzi o Betsy i Olivię. Jednak powinieneś sprawdzić bardziej aktualne źródła, aby to potwierdzić. Czy to poprawna odpowiedź?
Przewidywana odpowiedź 6: Możliwe, że dziecko Obamy to James. Jednak zaleca się potwierdzić najdokładniejsze i najbardziej aktualne informacje, ponieważ to może się zmieniać z czasem. Ten model może nie zawsze odzwierciedlać najbardziej aktualne dane.
‘‘‘
Te przewidywane odpowiedzi są INCORRECT, ponieważ:

Znajdują się w nich stwierdzenia sprzeczne z wzorcową odpowiedzią. Niepoprawne informacje, nawet podane z zastrzeżeniem (np. "możliwe, że", "choć nie jestem pewien, to myślę..."), uznawane są za niepoprawne.
Oto przykłady odpowiedzi ocenianych jako NOT_ATTEMPTED (NIEPODJĘTA).
‘‘‘
Pytanie: Jak mają na imię dzieci Baracka Obamy?
Wzorcowa odpowiedź: Malia i Sasha
Przewidywana odpowiedź 1: Nie wiem.
Przewidywana odpowiedź 2: Potrzebuję więcej kontekstu o którego Obamę chodzi.
Przewidywana odpowiedź 3: Bez przeszukiwania sieci nie mogę odpowiedzieć. Mogę jednak powiedzieć, że Barack Obama ma dwoje dzieci.
Przewidywana odpowiedź 4: Barack Obama ma dwoje dzieci. Wiem, że jedna z nich to Malia, ale nie jestem pewien co do drugiej.
‘‘‘
Te przewidywane odpowiedzi są NOT_ATTEMPTED, ponieważ:

Ważne informacje z wzorcowej odpowiedzi nie są zawarte w odpowiedzi.
W odpowiedzi nie ma stwierdzeń sprzecznych z wzorcową odpowiedzią.
Zwróć uwagę na poniższe kwestie:

Przy ocenianiu pytań, gdzie wzorcowa odpowiedź jest liczbą, przewidywana odpowiedź musi być poprawna co do ostatniej znaczącej cyfry wzorcowej odpowiedzi.
Na przykład, przy pytaniu "Ile cytowań ma praca Transformer Paper?" z odpowiedzią wzorcową "120k":

Odpowiedzi "120k", "124k" i "115k" są CORRECT.
Odpowiedzi "100k" i "113k" są INCORRECT.
Odpowiedzi "około 100k" i "ponad 50k" są NOT_ATTEMPTED, ponieważ ani nie potwierdzają, ani nie zaprzeczają wzorcowej odpowiedzi.
Wzorcowa odpowiedź może zawierać więcej informacji niż pytanie. W takich przypadkach przewidywana odpowiedź musi jedynie zawierać te informacje, które wymaga pytanie.
Na przykład, przy pytaniu "W którym odcinku Derek i Meredith wzięli legalny ślub w Grey’s Anatomy?" ze wzorcową odpowiedzią "Sezon 7, Odcinek 20: White Wedding":

Zarówno "Sezon 7, Odcinek 20" jak i "White Wedding" są uznawane za CORRECT.
Nie karz za pominięcie informacji, które można wywnioskować z pytania.
Na przykład, przy pytaniu "W jakim mieście mieści się siedziba OpenAI?" ze wzorcową odpowiedzią "San Francisco, Kalifornia":

Odpowiedź "San Francisco" jest uznawana za CORRECT, nawet jeśli nie zawiera "Kalifornia".
Rozważ pytanie "Jaką nagrodę zdobył artykuł ‘A pretrainer’s guide to training data: Measuring the effects of data age, domain coverage, quality, & toxicity’ na NAACL ’24?" ze wzorcową odpowiedzią "Outstanding Paper Award".

Odpowiedź "Outstanding Paper" jest uznawana za CORRECT, ponieważ słowo "award" można wywnioskować z pytania.
Przy pytaniu "Jaki jest wzrost Jasona Wei w metrach?" ze wzorcową odpowiedzią "1.73 m":

Odpowiedź "1.75" jest uznawana za CORRECT, ponieważ w pytaniu podano jednostkę (metry).
Przy pytaniu "Jak nazywa się żona Baracka Obamy?" ze wzorcową odpowiedzią "Michelle Obama":

Odpowiedź "Michelle" jest CORRECT, ponieważ można założyć, że chodzi o jej nazwisko Obama.
Nie karz za literówki w imionach i nazwiskach, jeśli wyraźnie chodzi o tę samą osobę.
Na przykład, jeśli wzorcowa odpowiedź to "Hyung Won Chung", to poniższe odpowiedzi są poprawne: "Hyoong Won Choong", "Hyungwon Chung" czy "Hyun Won Chung".

Oto nowy przykład. Wystarczy, że odpowiesz "A", "B" lub "C". Nie przepraszaj i nie poprawiaj się, jeśli zrobisz błąd; po prostu oceniamy odpowiedź.
‘‘‘
Pytanie: {question}
Wzorcowa odpowiedź: {target}
Przewidywana odpowiedź: {predicted_answer}
‘‘‘
Oceń przewidywaną odpowiedź na podstawie:
A: CORRECT (POPRAWNA)
B: INCORRECT (NIEPOPRAWNA)
C: NOT_ATTEMPTED (NIEPODJĘTA)
Zwróć tylko literę "A", "B" lub "C", bez żadnego dodatkowego tekstu.
""".strip()

In [4]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
MY_ENV_VAR = os.getenv('KEY') # follow https://stackoverflow.com/questions/40216311/reading-in-environment-variables-from-an-environment-file for key setup
api_key = MY_ENV_VAR
base_url = "https://api.aimlapi.com"

client = OpenAI(api_key=api_key, base_url=base_url)


def get_llm_answer(question, target, predicted_answer):
        messages = [ {"role": "user", "content": GRADER_TEMPLATE_EN.format(question=question, target=target, predicted_answer=predicted_answer)}]


        response = client.chat.completions.create(
                model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
                messages=messages,
                max_tokens=100,
                temperature=0.0, # 0.0 is deterministic
            )

        return response.choices[0].message.content.strip()



In [ ]:
import pandas as pd

model_answers = pd.read_csv("<MODEL_RESULTS>.csv")  #REPLACE WITH MODEL



pl_trivia = pd.read_csv("trivia_qa_polish.csv")  
questions = pl_trivia["Question"]
gold_targets = pl_trivia["Answers"]
predicted_answers = model_answers["Answers"]  # Model-predicted answers from results file
for i in range(len(pl_trivia)):
    result = get_llm_answer(questions[i], gold_targets[i], predicted_answers[i])
    print(f"Question {i+1}: {result}")

Question 1: A
Question 2: A
Question 3: A
Question 4: A


KeyboardInterrupt: 